In [1]:
import numpy as np
import pandas as pd
from numpy import nan
import pickle
pd.set_option('display.max_rows', 200)

## Data Cleaning

### 1. load data

In [2]:
comprehensive = pd.read_stata('../Data/data/Comprehensive-Sample.dta')

In [3]:
comprehensive.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17920 entries, 0 to 17919
Data columns (total 80 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        17920 non-null  float32
 1   numcode                     17920 non-null  int16  
 2   oilreserves_full            14307 non-null  float32
 3   oilreserves                 12519 non-null  float32
 4   oilreserves_public          9950 non-null   float32
 5   newdiscovery_aspo           12354 non-null  float32
 6   aspo                        4736 non-null   float32
 7   wildcat                     12354 non-null  float32
 8   endowment                   14160 non-null  float32
 9   pop_maddison                10441 non-null  float64
 10  ecgrowth                    9341 non-null   float32
 11  efrac                       390 non-null    float32
 12  lfrac                       390 non-null    float32
 13  rfrac                       390

In [23]:
aspo = pd.read_stata('../Data/data/ASPO-Sample.dta')

In [4]:
'discoveryaspoPC' in aspo.columns

True

In [17]:
aspo[['wildcat','year']][~aspo['wildcat'].isna()].query('wildcat > 0')

,wildcat,year
1,1.0,1930.0
28,1.0,1957.0
31,1.0,1960.0
34,3.0,1963.0
35,1.0,1964.0
...,...,...
5097,10.0,1986.0
5098,3.0,1987.0
5099,4.0,1988.0
5100,16.0,1989.0


### 2. function to do transformation

In [48]:
# Create a sample dataframe
df = pd.DataFrame({"country": np.random.choice(["A", "B", "C"], size=20),
                   "year": np.arange(20),
                   "lognum1": np.random.randint(1, 4, size=20),
                   "num2": np.random.randint(20000, 90000, size=20),
                   "num3": np.random.randint(20000, 90000, size=20),
                   "num4": np.random.randint(20000, 90000, size=20)})

In [49]:
df = df.sort_values(by=['country','year'], ascending=True)

In [50]:
df

,country,year,lognum1,num2,num3,num4
0,A,0,1,53289,84438,33081
5,A,5,3,89634,28144,49934
6,A,6,2,89617,58367,30182
10,A,10,1,76584,45595,79870
14,A,14,3,61607,78038,77474
18,A,18,3,89319,79395,24936
19,A,19,2,82841,75240,33007
1,B,1,2,72677,59579,65416
2,B,2,1,79880,24483,52615
3,B,3,2,32197,33476,87753


In [4]:
def transform_df(df, groupby_cols, log_cols, diff_cols, pct_change_cols, binarize_cols):
    
    new_df = df.copy()
    new_cols = []

    for i, col_name in enumerate(log_cols):
        new_col_name = col_name[3:] # to get rid of 'log' in the column name
        new_df[new_col_name] = np.exp(new_df[col_name])
        log_cols[i] = new_col_name

    all_cols = log_cols + pct_change_cols
    
    for col_name in all_cols:
        
        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df.groupby(groupby_cols)[col_name].pct_change()
        new_cols.append(diff_name)

        if col_name in binarize_cols:
            binarize_name = col_name + '_binarize'
            new_df[binarize_name] = np.NAN
            new_df.loc[new_df[diff_name] > 0.1, binarize_name] = 1
            new_df.loc[new_df[diff_name] <= 0.1, binarize_name] = 0
            new_df.loc[pd.isnull(new_df[diff_name]), binarize_name] = np.nan
            new_cols.append(binarize_name)

    for col_name in diff_cols:

        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df[col_name] - new_df.groupby(groupby_cols)[col_name].shift(1)
        new_cols.append(diff_name)
            
    return new_df[new_cols]

In [56]:
transform_df(df, ['country'], ['lognum1'], ['num4'], ['num2','num3'], ['num3'])

,num1_diff,num2_diff,num3_diff,num3_binarize,num4_diff
0,NaN,NaN,NaN,NaN,NaN
5,6.389056,0.682036,-0.666690,0.0,16853.0
6,-0.632121,-0.000190,1.073870,1.0,-19752.0
10,-0.632121,-0.145430,-0.218822,0.0,49688.0
14,6.389056,-0.195563,0.711547,1.0,-2396.0
18,0.000000,0.449819,0.017389,0.0,-52538.0
19,-0.632121,-0.072527,-0.052333,0.0,8071.0
1,NaN,NaN,NaN,NaN,NaN
2,-0.632121,0.099110,-0.589067,0.0,-12801.0
3,1.718282,-0.596933,0.367316,1.0,35138.0


--

### 3. comprehensive - cleaning

In [45]:
v_lst = ['onset2COWCS',
         'onsetUCS',
         'coup',
         'periregular',
         'milexp_pergdpSIPRI',
         #'logmilexgdpSIPRI',
         'numcode',
         'year',
         
         #'logGDP_M',
         'ecgrowth',
         'pop_maddison',
         #'logpop_M',
         'logpopdens', ####
         'democracy',
         'logmountain',
         'ethnic_fractionalization',
         'religion_fractionalization',
         'language_fractionalization',
         'leg_british',
         'no_transition',
         'wildcat',
         
         'valoilres', # value of oil reserves
         #'logvaloilres',
         'valoilres_public',
         #'logvaloilres_public',
         'oilpop',
         #'logoilres',
         'valoilres_impute',
         #'logvaloilres_impute',
         'oilpop_impute',
         #'logoilres_impute',
         'out_regdisaster']
         #'logoutreg'

In [62]:
# variables that need transformation
# transform_lst = ['logmilexgdpSIPRI', # defense burden
#                 'logpop_M',
#                 'logpopdens',
#                 'democracy', # democracy index
#                 'wildcat', # wildcat drilling 
#                 'logoutreg', # out of region disaster 
#                 'logvaloilres',
#                 'logvaloilres_public',
#                 'logoilres',
#                 'logvaloilres_impute',
#                 'logoilres_impute'
#                 ]

transform_lst = ['milexp_pergdpSIPRI',
                'pop_maddison',
                'logpopdens',
                'democracy',
                'wildcat',
                'out_regdisaster',
                'valoilres', # value of oil reserves
                'valoilres_public',
                'oilpop',
                'valoilres_impute',
                'oilpop_impute']

In [5]:
# variables that need to take exp and then compute pct change
log_lst = ['logpopdens']

# variables that need to take diff directly
diff_lst = ['milexp_pergdpSIPRI']

# variables that need to take pct change directly
pct_change_lst = ['pop_maddison', 
                'democracy',
                'wildcat',
                'out_regdisaster',
                'valoilres', # value of oil reserves
                'valoilres_public', # value of oil reserves from public data
                'oilpop', # oil reserves per capita in million barrels per 1000 persons
                'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
                'oilpop_impute'] # oil reserves per capita - imputed

# a subset of pct_change_lst
# variables that need to be binarized
binarize_lst = ['valoilres', # value of oil reserves
              'valoilres_public', # value of oil reserves from public data
              'oilpop', # oil reserves per capita in million barrels per 1000 persons
              'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
              'oilpop_impute'] # oil reserves per capita - imputed

# variables that don't need transformation
untransformed_lst = ['onset2COWCS',
                    'onsetUCS',
                    'coup',
                    'periregular',
                    'numcode',
                    'year',
                    'ecgrowth',
                    'logmountain',
                    'ethnic_fractionalization',
                    'religion_fractionalization',
                    'language_fractionalization',
                    'leg_british',
                    'no_transition']

In [6]:
comprehensive = comprehensive.replace(0, 0.1**5)

In [7]:
comprehensive['onset2COWCS'] = comprehensive.onset2COWCS.replace(0.1**5, 0)
comprehensive['onsetUCS'] = comprehensive.onsetUCS.replace(0.1**5, 0)
comprehensive['leg_british'] = comprehensive.leg_british.replace(0.1**5, 0)

In [8]:
comprehensive = comprehensive.sort_values(by=['numcode','year'], ascending=True)

In [9]:
new_df = transform_df(df = comprehensive,
                      groupby_cols = ['numcode'], 
                      log_cols = log_lst, 
                      diff_cols = diff_lst, 
                      pct_change_cols = pct_change_lst, 
                      binarize_cols = binarize_lst)
                      

In [25]:
comprehensive_new = pd.concat([comprehensive[untransformed_lst], new_df], axis = 1)

In [26]:
comprehensive_new.shape

(17920, 29)

In [13]:
comprehensive_new.columns

Index(['onset2COWCS', 'onsetUCS', 'coup', 'periregular', 'numcode', 'year',
       'ecgrowth', 'logmountain', 'ethnic_fractionalization',
       'religion_fractionalization', 'language_fractionalization',
       'leg_british', 'no_transition', 'popdens_diff', 'pop_maddison_diff',
       'democracy_diff', 'wildcat_diff', 'out_regdisaster_diff',
       'valoilres_diff', 'valoilres_binarize', 'valoilres_public_diff',
       'valoilres_public_binarize', 'oilpop_diff', 'oilpop_binarize',
       'valoilres_impute_diff', 'valoilres_impute_binarize',
       'oilpop_impute_diff', 'oilpop_impute_binarize',
       'milexp_pergdpSIPRI_diff'],
      dtype='object')

In [7]:
# potential selection of variables
len((comprehensive_new[['onset2COWCS',
                   'valoilres_binarize',
                   'ecgrowth',
                   'pop_maddison_diff',
                   'popdens_diff',
                   'democracy_diff',
                   'logmountain',
                   'ethnic_fractionalization',
                   'religion_fractionalization',
                   'language_fractionalization',
                   'leg_british',
                   'numcode',
                   'year']].dropna())['numcode'].unique())

103

In [32]:
#comprehensive_new[['oilpop_impute_binarize','year']].groupby('oilpop_impute_binarize').count()

,year
oilpop_impute_binarize,
0.0,7527
1.0,398


In [27]:
comprehensive_new.to_pickle("../Data/data/comprehensive_new.pkl")  

In [48]:
comprehensive_new['valoilres_binarize'].describe()

count    7873.000000
mean        0.111266
std         0.314482
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: valoilres_binarize, dtype: float64

In [43]:
comprehensive['valoilres_impute'].describe()

count     7620.000000
mean       124.457787
std        849.155640
min          0.000010
25%          0.000010
50%          0.000010
75%          3.599900
max      21176.099609
Name: valoilres_impute, dtype: float64

In [47]:
(comprehensive_new['valoilres_impute_binarize'] == comprehensive_new['valoilres_binarize']).sum()

7650

In [ ]:
for col_name in comprehensive_new.columns:
    print(comprehensive_new[col_name].describe())

In [2]:
comprehensive_new = pd.read_pickle('../Data/data/comprehensive_new.pkl')

In [5]:
len(comprehensive_new['numcode'].unique())

224

In [10]:
comprehensive_new_full = pd.concat([comprehensive, new_df], axis = 1)

In [11]:
comprehensive_new_full.to_pickle("../Data/data/comprehensive_new_full.pkl")  

### 4. ASPO - cleaning

In [6]:
aspo.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5120 entries, 0 to 5119
Data columns (total 108 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    year                        float32
 1    numcode                     int16  
 2    oilreserves_full            float32
 3    oilreserves                 float32
 4    oilreserves_public          float32
 5    newdiscovery_aspo           float32
 6    imputedzero                 float32
 7    aspo                        float32
 8    wildcat                     float32
 9    endowment                   float32
 10   ecgrowth                    float32
 11   country                     object 
 12   incidence2COW               float64
 13   onset2COWCS                 float32
 14   incidenceU                  float32
 15   onsetUCS                    float32
 16   Fearon_war                  float64
 17   onset_FearonCS              float32
 18   Sambanis_war                float64
 19   onse

In [33]:
def transform_df(df, groupby_cols, log_cols, log_diff_cols, pct_change_cols, binarize_cols):
    
    new_df = df.copy()
    new_cols = []

    for i, col_name in enumerate(log_cols):
        new_col_name = col_name[3:] # to get rid of 'log' in the column name
        new_df[new_col_name] = np.exp(new_df[col_name])
        log_cols[i] = new_col_name

    for col_name in log_diff_cols:
        
        new_col_name = col_name[3:]
        new_df[new_col_name] = np.exp(new_df[col_name])
        diff_name = new_col_name + '_diff'
        new_df[diff_name] = new_df[new_col_name] - new_df.groupby(groupby_cols)[new_col_name].shift(1)
        new_cols.append(diff_name)

    all_cols = log_cols + pct_change_cols
    
    for col_name in all_cols:
        
        diff_name = col_name + '_diff'
        new_df[diff_name] = new_df.groupby(groupby_cols)[col_name].pct_change()
        new_cols.append(diff_name)

        if col_name in binarize_cols:
            binarize_name = col_name + '_binarize'
            new_df[binarize_name] = np.NAN
            new_df.loc[new_df[diff_name] > 0.1, binarize_name] = 1
            new_df.loc[new_df[diff_name] <= 0.1, binarize_name] = 0
            new_df.loc[pd.isnull(new_df[diff_name]), binarize_name] = np.nan
            new_cols.append(binarize_name)
            
    return new_df[new_cols]

In [34]:
# variables that need to take exp and then compute pct change
log_lst = ['logpopdens',
          'logpop_M',
          'logvaloilres',
          'logvaloilres_impute'
          ]

# variables that need to take diff directly
log_diff_lst = ['logmilexgdpSIPRI']

# variables that need to take pct change directly
pct_change_lst = ['democracy',
                'wildcat',
                'out_regdisaster',
                
                
                
                'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
                'oilpop_impute'] # oil reserves per capita - imputed

# a subset of pct_change_lst
# variables that need to be binarized
binarize_lst = ['valoilres', # value of oil reserves
              
              
              'valoilres_impute', # value of oilpop_impute (multiply by crude oil price)
              'oilpop_impute'] # oil reserves per capita - imputed

# variables that don't need transformation
untransformed_lst = ['onset2COWCS',
                    'onsetUCS',
                    'coup',
                    'periregular',
                    'numcode',
                    'year',
                    'ecgrowth',
                    'logmountain',
                    'ethnic_fractionalization',
                    'religion_fractionalization',
                    'language_fractionalization',
                    'leg_british',
                    'no_transition']

In [24]:
aspo = aspo.replace(0, 0.1**5)

In [17]:
aspo['onset2COWCS'] = aspo.onset2COWCS.replace(0.1**5, 0)
aspo['onsetUCS'] = aspo.onsetUCS.replace(0.1**5, 0)
aspo['leg_british'] = aspo.leg_british.replace(0.1**5, 0)

In [18]:
aspo = aspo.sort_values(by=['numcode','year'], ascending=True)

In [35]:
new_df_aspo = transform_df(df = aspo,
                        groupby_cols = ['numcode'], 
                        log_cols = log_lst, 
                        log_diff_cols = log_diff_lst, 
                        pct_change_cols = pct_change_lst, 
                        binarize_cols = binarize_lst)
                      

In [39]:
new_df_aspo.columns

Index(['milexgdpSIPRI_diff', 'popdens_diff', 'pop_M_diff', 'valoilres_diff',
       'valoilres_binarize', 'valoilres_impute_diff',
       'valoilres_impute_binarize', 'democracy_diff', 'wildcat_diff',
       'out_regdisaster_diff', 'valoilres_impute_diff',
       'valoilres_impute_binarize', 'oilpop_impute_diff',
       'oilpop_impute_binarize'],
      dtype='object')

In [43]:
aspo_new = pd.concat([aspo[untransformed_lst], new_df_aspo], axis = 1)

In [44]:
aspo_new.to_pickle("../Data/data/aspo_new.pkl")  

In [45]:
aspo_full = pd.concat([aspo, new_df_aspo], axis = 1)

In [46]:
aspo_full.to_pickle("../Data/data/aspo_full.pkl") 

In [5]:
aspo_full = pd.read_pickle('../Data/data/aspo_full.pkl')

In [20]:
aspo_new = pd.read_pickle('../Data/data/aspo_new.pkl')

In [22]:
aspo_new[['wildcat_diff','year']][~aspo_new['wildcat_diff'].isna()].query('wildcat_diff > 0')

,wildcat_diff,year
28,99999.000000,1957.0
31,99999.000000,1960.0
34,299999.000000,1963.0
41,199999.000000,1970.0
48,99999.000000,1977.0
...,...,...
5092,0.285714,1981.0
5093,0.222222,1982.0
5096,0.888889,1985.0
5099,0.333333,1988.0


In [6]:
'discoveryaspoPC' in aspo_full.columns

True

In [13]:
aspo_full['wildcat_diff'].isna().sum()

555

In [27]:
aspo_full[['wildcat_diff','year']][~aspo_full['wildcat_diff'].isna()].query('wildcat_diff > 0')

,wildcat_diff,year
28,99999.000000,1957.0
31,99999.000000,1960.0
34,299999.000000,1963.0
41,199999.000000,1970.0
48,99999.000000,1977.0
...,...,...
5092,0.285714,1981.0
5093,0.222222,1982.0
5096,0.888889,1985.0
5099,0.333333,1988.0


In [19]:
aspo_full

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,imputedzero,aspo,wildcat,endowment,...,valoilres_binarize,valoilres_impute_diff,valoilres_impute_binarize,democracy_diff,wildcat_diff,out_regdisaster_diff,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize
0,1929.0,8,0.281900,0.281900,NaN,NaN,NaN,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930.0,8,0.282660,0.282660,NaN,0.00076,NaN,1.0,1.00000,0.8,...,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,-0.002409,0.0
2,1931.0,8,0.282660,0.282660,NaN,0.00001,NaN,1.0,0.00001,0.8,...,0.0,-0.005175,0.0,0.0,-0.99999,9.525810e+11,-0.005175,0.0,-0.006073,0.0
3,1932.0,8,0.282660,0.282660,NaN,0.00001,NaN,1.0,0.00001,0.8,...,0.0,0.003964,0.0,0.0,0.00000,-1.000000e+00,0.003964,0.0,-0.005035,0.0
4,1933.0,8,0.282649,0.282649,NaN,0.00001,NaN,1.0,0.00001,0.8,...,0.0,-0.002133,0.0,0.0,0.00000,3.566128e+10,-0.002133,0.0,-0.005049,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,2004.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.00000,7.267240e-01,0.000000,0.0,0.000000,0.0
5116,2005.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.00000,1.625349e+00,0.000000,0.0,0.000000,0.0
5117,2006.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.00000,-8.715621e-01,0.000000,0.0,0.000000,0.0
5118,2007.0,890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.0,0.0,0.00000,1.888971e-01,0.000000,0.0,0.000000,0.0
